# Extract BERT

Notebook to experiment with libraries to extract parts of BERT embeddings

In [1]:
import transformers
import torch

I1031 15:17:35.353595 139773181261568 file_utils.py:39] PyTorch version 1.2.0 available.
I1031 15:17:36.716568 139773181261568 modeling_xlnet.py:194] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [ ]:
bert_model = transformers.BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
bert_tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

In [3]:
STR = "My cat is called Xiaonuanhuo and she is warm and fluffy"
bert_tokenizer.tokenize(STR)

['my',
 'cat',
 'is',
 'called',
 'xiao',
 '##nu',
 '##an',
 '##hu',
 '##o',
 'and',
 'she',
 'is',
 'warm',
 'and',
 'fluffy']

In [4]:
input_ids = bert_tokenizer.encode(STR)
input_ids

[2026,
 4937,
 2003,
 2170,
 19523,
 11231,
 2319,
 6979,
 2080,
 1998,
 2016,
 2003,
 4010,
 1998,
 27036]

In [5]:
len(input_ids)

15

In [6]:
input_ids2 = bert_tokenizer.encode("this is a much shorter sentence")
len(input_ids2)

6

In [7]:
bert_embeddings = bert_model(torch.tensor([input_ids]))
#bert_embeddings = bert_model(torch.tensor([input_ids, input_ids2]))
bert_embeddings[0]

tensor([[[-0.4095,  0.0730,  0.0156,  ...,  0.0193,  0.1457,  0.6264],
         [-0.6220, -0.3098,  0.4594,  ..., -0.2029,  0.3284,  1.0646],
         [-1.0167, -0.3767,  0.4307,  ..., -0.0539,  0.4236,  0.9048],
         ...,
         [-0.7988,  0.2165,  0.7978,  ..., -0.0076,  0.0286,  0.4517],
         [-0.7674, -0.0174,  0.5640,  ...,  0.0461,  0.0452,  0.5110],
         [-0.7672,  0.1120,  0.8036,  ...,  0.1290,  0.1001,  0.5393]]],
       grad_fn=<NativeLayerNormBackward>)

In [8]:
bert_embeddings[0].shape

torch.Size([1, 15, 768])

In [9]:
# Final hidden layer, one for each token
bert_embeddings[0].shape

torch.Size([1, 15, 768])

In [10]:
# Pooled layer
bert_embeddings[1].shape

torch.Size([1, 768])

In [11]:
len(bert_embeddings[2])

13

In [12]:
# Nth hidden layer of BERT
layer = 7
bert_embeddings[2][layer].shape

torch.Size([1, 15, 768])